In [2]:
!pip install datasets evaluate accelerate
!pip install git+https://github.com/huggingface/transformers

^C


Connect Google Drive and load the training file.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copy("/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/run_glue_no_trainer_modded.py", "/content/run_glue_no_trainer_modded.py")

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import os
import torch
print(f'torch Version {torch.__version__}')
import transformers
print(f'transformers Version {transformers.__version__}')

torch Version 2.0.0
transformers Version 4.31.0.dev0


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\Valentin\AppData\Local\Temp\pip-req-build-m_k__rwd'


  Cloning https://github.com/huggingface/transformers to c:\users\valentin\appdata\local\temp\pip-req-build-m_k__rwd
  Resolved https://github.com/huggingface/transformers to commit 4f85aaa6c9714a6b2bbc2cdfd2399d9e4ace966f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


Declare all possible parameters to be examinated.

In [4]:
possible_values = {
    "task" : ['wnli', 'rte'],
    "learning_rate" : [5e-5, 1e-5],
    "hidden_dropout" : [0.1, 0.5, 0.8],
    "use_pretrain_dropout_in_first_pass" : [True, False],
    "train_data_frac" : [1, 0.5, 0.1],
    "use_modded" : [True, False],
    "num_train_epochs" : [5, 10, 20]
}

Create an exhaustive list of all parameter combinations.

In [5]:
from itertools import product
keys, values = zip(*possible_values.items())
permutations_dicts = [dict(zip(keys, v)) for v in product(*values)]
print(len(permutations_dicts))
permutations_dicts

432


[{'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': True,
  'num_train_epochs': 5},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': True,
  'num_train_epochs': 10},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': True,
  'num_train_epochs': 20},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': False,
  'num_train_epochs': 5},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropout': 0.1,
  'use_pretrain_dropout_in_first_pass': True,
  'train_data_frac': 1,
  'use_modded': False,
  'num_train_epochs': 10},
 {'task': 'wnli',
  'learning_rate': 5e-05,
  'hidden_dropou

Load the Tensorboard for live tracking.

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/runs/"

Launching TensorBoard...

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "G:\\Meine Ablage\\Masterarbeit\\fantastic-umbrella\\runs"

In [1]:
def compose_instruction(task,
                        output_dir,
                        num_train_epochs = 3,
                        learning_rate = 1e-5,
                        train_data_frac = 1,
                        hidden_dropout = None,
                        use_modded = False,
                        use_pretrain_dropout_in_first_pass = False,
                        model_name_or_path="bert-base-cased",
                        max_length = 128,
                        per_device_train_batch_size = 32,
                        seed = 0):
    

    base_command = 'python run_glue_no_trainer_modded.py '
    argument_list = ["--model_name_or_path", model_name_or_path,
                     "--max_length", max_length,
                     "--per_device_train_batch_size", per_device_train_batch_size,
                     "--seed", seed,
                     "--with_tracking",
                     "--report_to", "tensorboard"
                    ]



    argument_list = argument_list + ["--task_name", f'{task}']
    argument_list = argument_list + ["--num_train_epochs", f'{num_train_epochs}']
    argument_list = argument_list + ["--learning_rate", f'{learning_rate}']
    argument_list = argument_list + ["--train_data_frac", f'{train_data_frac}']
    argument_list = argument_list + ["--hidden_dropout", f'{hidden_dropout}']


    if use_modded:
        argument_list.append("--use_modded")

    if use_pretrain_dropout_in_first_pass:
        argument_list.append("--use_pretrain_dropout_in_first_pass")

    argument_list = argument_list + ["--output_dir",output_dir]
    argument_string = ' '.join(argument_list)
    instruction = base_command + argument_string
    return instruction

Create an overview of commands

Construct and execute the script command for every parameter combination.

In [7]:
import datetime
import os
for iteration, parameters in enumerate(permutations_dicts):
    print(f'##################################### Current iteration: {iteration} #####################################')

    folder_name = 'run-{date:%Y%m%d_%H%M%S}'.format( date=datetime.datetime.now() )
    output_dir = f"/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/runs/{folder_name}"


    instruction = compose_instruction(task=parameters["task"],
                                        output_dir=output_dir,
                                        num_train_epochs=parameters["num_train_epochs"],
                                        learning_rate=parameters["learning_rate"],
                                        train_data_frac=parameters["train_data_frac"],
                                        hidden_dropout=parameters["hidden_dropout"],
                                        use_modded=parameters["use_modded"],
                                        use_pretrain_dropout_in_first_pass=parameters["use_pretrain_dropout_in_first_pass"],
                                        model_name_or_path="bert-base-cased"
                                        )
    
    

    # run the script with the desired arguments
    # os.system(instruction) # vscode style
    !{instruction} # colab style

##################################### Current iteration: 0 #####################################


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "c:\Users\Valentin\miniconda3\envs\fantastic-umbrella\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "c:\Users\Valentin\miniconda3\envs\fantastic-umbrella\lib\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 